In [1]:
import requests 

In [2]:
import docx
import spacy
import torch
import numpy as np


# Load spaCy Russian tokenizer
nlp = spacy.blank("ru")



def read_docx(file_path):
    # Open the Word document
    doc = docx.Document(file_path)
    
    # Extract text content from the document
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    
    # Join all paragraphs into a single string
    document_text = "\n".join(full_text)
    return document_text

def chunking(document_text, max_chunk_len=600):
    # Split the document into chunks
    doc = nlp(document_text)
    chunks = []
    current_chunk = []

    for token in doc:
        if len(current_chunk) + len(token.text) > max_chunk_len:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

        current_chunk.append(token.text)

    if current_chunk:
        chunks.append(" ".join(current_chunk))



    return chunks


file_path = "main_doc.docx"
document_text = read_docx(file_path)
chunks = chunking(document_text)

In [3]:
import numpy as np





def jaccard_similarity(s1, s2):
    s1 = s1.lower().split(" ")
    s2 = s2.lower().split(" ")
    set1, set2 = set(s1), set(s2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union




def jacc_search(query, chunks):
    distances = []

    for chunk in chunks:
        dist = jaccard_similarity(query, chunk)
        distances.append(dist)

    # Find indices of the top 3 closest chunks 
    closest_indices = np.argsort(distances)[::-1][:3]
    closest_chunks = [chunks[idx] for idx in closest_indices]

    return closest_chunks, distances,closest_indices



query = 'За что отвечает академический руководитель?'
closest_chunks,dist,cl = jacc_search(query,chunks)
i=1
print(dist)
print(dist[cl[0]])
print()
print(cl)
print("Closest chunks to the query:")
for idx, chunk in enumerate(closest_chunks, start=1):
    print('chunk ',i)
    print()
    i+=1
    print(f"{idx}. {chunk}")
    print()



[0.007168458781362007, 0.0036496350364963502, 0.0, 0.0034482758620689655, 0.0034965034965034965, 0.0035087719298245615, 0.007380073800738007, 0.0038461538461538464, 0.007142857142857143, 0.003389830508474576, 0.006968641114982578, 0.0036231884057971015, 0.007042253521126761, 0.007407407407407408, 0.006514657980456026, 0.006944444444444444, 0.0036900369003690036, 0.0, 0.0034129692832764505, 0.0033222591362126247, 0.003278688524590164, 0.003205128205128205, 0.0037313432835820895, 0.0, 0.0036231884057971015, 0.003484320557491289, 0.0035087719298245615, 0.0031847133757961785, 0.00398406374501992, 0.011764705882352941, 0.007272727272727273, 0.003676470588235294, 0.007575757575757576, 0.0035842293906810036, 0.0036101083032490976, 0.005434782608695652, 0.003289473684210526, 0.007575757575757576, 0.0034965034965034965, 0.003937007874015748, 0.003787878787878788, 0.0038910505836575876, 0.0]
0.011764705882352941

[29 32 37]
Closest chunks to the query:
chunk  1

1. НИУ ВШЭ _ _ _ _ _ _ _ _ _ _ _ 

In [4]:
context = str(closest_chunks)

prompt = {
    "modelUri": "gpt://folder-id/yandexgpt-lite",
    "completionOptions": {
        "stream": False,
        "temperature": 0.6,
        "maxTokens": "2000"
    },
    "messages": [
        {
            "role": "system",
            "text": "ты умный помошник студента, используй документы ниже, чтобы ответить на поставленный вопрос, если в документах этой информации нет, то так и скажи \
             вот ДОКУМЕНТЫ: " + context
        },
        {
            "role": "user",
            "text": query
        }
    ]
}


url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Api-Key your key"
}

response = requests.post(url, headers=headers, json=prompt)
result = response.text
response_data = response.json()
assistant_text = response_data['result']['alternatives'][0]['message']['text']
print(assistant_text)

Согласно приведённому вами документу, **академический руководитель образовательной программы** на факультете экономических наук **принимает решение о переносе срока сдачи НЭ по ЦК** для конкретного студента. Таким образом, исходя из предоставленной информации, **ответ на вопрос: «За перенос даты проведения независимого экзамена отвечает академический руководитель образовательной программы»**.
